# Marine Heatwave calculation 

Using this notebook to compute MHW diagnostics (duration and mean magnitude)

Applied to the OFAM future simulation with statics compute for various GWLs

Did the calculations with megamem


In [1]:
import xarray as xr
from dask.distributed import Client
import dask.array as da
from dask import delayed
import numpy as np
import os
import matplotlib.pyplot as plt
from glob import glob
import warnings

import dask
# Set configuration options
dask.config.set({
    'distributed.comm.timeouts.connect': '90s',  # Timeout for connecting to a worker
    'distributed.comm.timeouts.tcp': '90s',  # Timeout for TCP communications
})

In [2]:
client = Client()
client

2024-07-01 22:26:57,088 - distributed.preloading - INFO - Creating preload: /g/data/hh5/public/apps/dask-optimiser/schedplugin.py
2024-07-01 22:26:57,094 - distributed.utils - INFO - Reload module schedplugin from .py file
2024-07-01 22:26:57,103 - distributed.preloading - INFO - Import preload module: /g/data/hh5/public/apps/dask-optimiser/schedplugin.py


Modifying workers


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /node/gadi-mmem-bdw-0001.gadi.nci.org.au/38177/proxy/8787/status,
Dashboard: /node/gadi-mmem-bdw-0001.gadi.nci.org.au/38177/proxy/8787/status,Workers: 32
Total threads: 32,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33085,Workers: 32
Dashboard: /node/gadi-mmem-bdw-0001.gadi.nci.org.au/38177/proxy/8787/status,Total threads: 32
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:39137,Total threads: 1
Dashboard: /node/gadi-mmem-bdw-0001.gadi.nci.org.au/38177/proxy/32849/status,Memory: 0 B
Nanny: tcp://127.0.0.1:39013,


2024-07-01 22:27:56,450 - tornado.application - ERROR - Uncaught exception GET /individual-task-stream/ws (::1)
HTTPServerRequest(protocol='http', host='gadi-mmem-bdw-0001.gadi.nci.org.au:38177', method='GET', uri='/individual-task-stream/ws', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/tornado/web.py", line 1786, in _execute
    result = await result
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/tornado/websocket.py", line 272, in get
    await self.ws_connection.accept_connection(self)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/tornado/websocket.py", line 862, in accept_connection
    await self._accept_connection(handler)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/tornado/websocket.py", line 945, in _accept_connection
 

In [3]:
def drop_stuff(ds, coords_to_drop, vars_to_drop):
    ds = ds.drop_vars(coords_to_drop, errors='ignore')
    ds = ds.drop_vars(vars_to_drop, errors='ignore')
    return ds

In [4]:
def load_data(coords_to_drop, vars_to_drop, dir1_new, dir2_new):
    pattern0 = sorted(glob(dir1_new + 'ocean_temp_sfc*.nc'))
    pattern1 = sorted(glob(dir1_new + 'ocean_temp_sfc*200[0-5]*.nc'))
    pattern2 = sorted(glob(dir2_new + 'ocean_temp_sfc*.nc'))

    dsst1 = xr.open_mfdataset(pattern0, parallel=True, preprocess=lambda x: drop_stuff(x, coords_to_drop, vars_to_drop)).squeeze()
    dsst2 = xr.open_mfdataset(pattern1 + pattern2, parallel=True, preprocess=lambda x: drop_stuff(x, coords_to_drop, vars_to_drop)).squeeze()

    sst = dsst2.temp
    sst = sst.rename({"Time": "time"})
    ssth= dsst1.temp
    ssth= ssth.rename({"Time": "time"})
    return sst,ssth

In [5]:
def calculate_heatwave_metrics(sst, threshold90, period, start, end):
    sst_period = sst.sel(time=slice(start, end))
    nshape = np.shape(sst_period)
    nyears = int(nshape[0] / 365.25 + .5)

    expanded_threshold = threshold90.sel(dayofyear=sst_period['time'].dt.dayofyear)
    expanded_threshold = expanded_threshold.drop('dayofyear')

    diff = sst_period - expanded_threshold

    min_dur = 10
    hmask = sst_period > expanded_threshold
    hmask1 = hmask.rolling(time=min_dur, center=False).sum() >= min_dur
    hmask2 = hmask1.shift(time=((min_dur-1)*-1), fill_value=False)
    hmask3 = hmask1 | hmask2

    dur_mhw = (hmask3.sum('time') / nyears)
    mag_mhw = ((hmask3 * diff).mean('time'))

    dur_mhw = dur_mhw.rename('duration').assign_attrs(units='days per year')
    mag_mhw = mag_mhw.rename('magnitude').assign_attrs(units='degrees Celsius')

    result = xr.merge([dur_mhw, mag_mhw])

    return result

In [6]:
def save_results(result_ds, period):
    file_path = f"/g/data/ia39/ncra/ocean/{period}_mhw.nc"
    result_ds.to_netcdf(file_path)
    print(f"Saved results to {file_path}")

In [7]:
def process_all_periods(GWL_periods, sst, ssth):
    for period, (start, end) in GWL_periods.items():
        if period == 'current':
            threshold90 = xr.open_dataset(f'/g/data/ia39/ncra/ocean/mhw/sst_percentile_daily_current.nc')['temp']
            threshold90 = threshold90.chunk({'dayofyear': 31})
            result_ds = calculate_heatwave_metrics(ssth, threshold90, period, start, end).compute()
        else:
            threshold90 = xr.open_dataset(f'/g/data/ia39/ncra/ocean/mhw/sst_percentile_daily_GW1p2.nc')['temp']
            threshold90 = threshold90.chunk({'dayofyear': 31})
            result_ds = calculate_heatwave_metrics(sst, threshold90, period, start, end).compute()
        
        save_results(result_ds, period)

In [8]:
def main():
    warnings.filterwarnings('ignore')

    coords_to_drop = ['st_edges_ocean', 'nv', 'st_ocean']
    vars_to_drop = ['Time_bounds', 'average_DT', 'average_T1', 'average_T2', 'st_ocean']

    dir1_new = '/g/data/fp2/OFAM3/jra55_historical.1/surface/'
    dir2_new = '/g/data/fp2/OFAM3/jra55_rcp8p5/surface/'

    GWL_periods = {
        'current': ('1995-01-01', '2014-12-31'),
        'GW1p2': ('2001-01-01', '2020-12-31'),
        'GW1p5': ('2015-01-01', '2034-12-31'),
        'GW2p0': ('2030-01-01', '2049-12-31'),
        'GW3p0': ('2053-01-01', '2072-12-31'),
        'GW4p0': ('2074-01-01', '2093-12-31')
    }

    GWL_periods = { 'current': ('1995-01-01', '2014-12-31')}
        
#    sst = load_data(coords_to_drop, vars_to_drop, dir1_new, dir2_new)
    sst,ssth = load_data(coords_to_drop, vars_to_drop, dir1_new, dir2_new)

    process_all_periods(GWL_periods, sst, ssth)

    client.close()

if __name__ == "__main__":
    main()

Saved results to /g/data/ia39/ncra/ocean/current_mhw.nc
